

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" /> 

# Laboratorio 5: Análisis Técnico y/o fundamental



### Equipo X

***Esteban Navarrete Garrido***


***José Alejandro Preciado Gandara***


***José Luis Suárez de Alba*** 


### Propósito: 

<div class=text-justify>  
Que puedas describir los efectos cuantitativos en el precio durante los comunicados de indicadores macroeconómicos, así como el detectar oportunidades de inversión a partir de los resultados obtenidos. Lo anterior, utilizando lo que se conoce como "Calendario Económico", y más en particular, aquel que incorpora información de las principales economías del mundo por lo que es ampliamiente utilizado en el mercado de Forex.
</div>

### Calendario económico:
&nbsp;
<div class=text-justify> 
**Retail salescontrol group:** Representa las ventas totales de la industria que se utilizan para preparar las estimaciones de los gastos del consumo personal para la mayoría de los productos
</div>

<div class=text-justify> 
**Retail sales ex Autos (MOM):** Representan los datos mensuales que muestran todos los productos vendidos por los minoristas en base a una muestra de tiendas minoristas de diferentes tipos y tamaños, excepto el sector automotriz.
</div>

### Importar librerias

In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import plotly.graph_objs as go
import chart_studio.plotly as py
from collections import Counter
from itertools import chain

### Importar datos

In [2]:
# precios = '/Users/Esteban/Desktop/MyST_Python/precios_historicos_eurusd.csv'
# calendario = '/Users/Esteban/Desktop/MyST_Python/calendario_economico.csv'
# precios = '/Users/preci/Documents/iteso/semestre 9/labs trading/Lab4/MyST-Lab5-EquipoX/precios_historicos_eurusd.csv'
# calendario = '/Users/preci/Documents/iteso/semestre 9/labs trading/Lab4/MyST-Lab5-EquipoX/calendario_economico.csv'
precios = '/Users/admin/Documents/9 Semestre/Microestructura y sistemas de trading/Proyecto 4/MyST-Lab5-EquipoX/precios_historicos_eurusd.csv'
calendario = '/Users/admin/Documents/9 Semestre/Microestructura y sistemas de trading/Proyecto 4/MyST-Lab5-EquipoX/calendario_economico.csv'

In [3]:
precios_df = pd.read_csv(precios, header=0, sep=',', parse_dates = False, skip_blank_lines = True)
precios_df.head()

,timestamp,open,high,low,close
0,2009-01-06 05:00:00,1.35564,1.35565,1.35383,1.35439
1,2009-01-06 05:05:00,1.35443,1.35489,1.35424,1.35474
2,2009-01-06 05:10:00,1.35480,1.35552,1.35439,1.35539
3,2009-01-06 05:15:00,1.35545,1.35549,1.35101,1.35101
4,2009-01-06 05:20:00,1.35101,1.35130,1.35021,1.35086


In [4]:
ce_df = pd.read_csv(calendario, header=0,sep=',', parse_dates=False, skip_blank_lines=True)
ce_df.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus
0,01/05/2009 09:30:00,EUR,Sentix Investor Confidence,European Monetary Union,1,-34.4,-42.3,NaN
1,01/05/2009 15:00:00,USD,Construction Spending (MoM),United States,1,-1.2,-0.4,-1.3
2,01/06/2009 08:55:00,EUR,Markit Services PMI,Germany,1,46.6,46.4,46.4
3,01/06/2009 09:00:00,EUR,Markit Services PMI,European Monetary Union,0,42.1,42.0,42.0
4,01/06/2009 10:00:00,EUR,Consumer Price Index (YoY),European Monetary Union,2,1.6,2.1,1.8


In [5]:
precios_df['timestamp'] = pd.to_datetime(precios_df['timestamp'])
precios_df['timestamp'] = precios_df['timestamp'].dt.tz_localize('UTC')
#precios_df.index = precios_df['timestamp'] ; precios_df = precios_df.drop('timestamp',axis=1)
precios_df.head()

,timestamp,open,high,low,close
0,2009-01-06 05:00:00+00:00,1.35564,1.35565,1.35383,1.35439
1,2009-01-06 05:05:00+00:00,1.35443,1.35489,1.35424,1.35474
2,2009-01-06 05:10:00+00:00,1.35480,1.35552,1.35439,1.35539
3,2009-01-06 05:15:00+00:00,1.35545,1.35549,1.35101,1.35101
4,2009-01-06 05:20:00+00:00,1.35101,1.35130,1.35021,1.35086


In [6]:
ce_df['timestamp'] = pd.to_datetime(ce_df['timestamp'])
ce_df['timestamp'] = ce_df['timestamp'].dt.tz_localize('UTC')
#ce_df.index = ce_df['timestamp'] ; ce_df = ce_df.drop('timestamp',axis=1)
ce_df.head()
ce_df.consensus.fillna(ce_df.previous, inplace= True)

In [7]:
Control = pd.DataFrame()
Autos   = pd.DataFrame()

for i in range(len(ce_df)):
    f = ce_df.index[i]
    if ce_df['Name'][i] == "Retail Sales Control Group":
        f=ce_df.index[i]
        Control[f] = ce_df.iloc[i]
    elif ce_df['Name'][i] == "Retail Sales ex Autos (MoM)":
        f=ce_df.index[i]
        Autos[f] = ce_df.iloc[i]
        
Autos   = Autos.T
Control = Control.T

df_indicadores = pd.concat([Autos, Control])

In [8]:
Autos   = Autos.reset_index(drop=True)
Control = Control.reset_index(drop=True) 

In [9]:
Autos['Volatility'] = Autos['Volatility'].astype(object).astype(int)
Autos.iloc[:,5:9]   = Autos.iloc[:,5:9].astype(object).astype(float)

Control['Volatility'] = Control['Volatility'].astype(object).astype(int)
Control.iloc[:,5:9]   = Control.iloc[:,5:9].astype(object).astype(float)

In [10]:
def escenario(d):
    d["escenario"]=d["actual"]
    for i in range(len(d)): 
        if d["actual"][i] >= d["consensus"][i] >= d["previous"][i]:
            d["escenario"][i] = "A"
        elif d["actual"][i] >= d["consensus"][i] < d["previous"][i]:
            d["escenario"][i]= "B"
        elif d["actual"][i] < d["consensus"][i] >= d["previous"][i]:
            d["escenario"][i]= "C"
        elif d["actual"][i] < d["consensus"][i] < d["previous"][i]:
            d["escenario"][i]= "D"  
    return d

In [11]:
Autos_esce          = escenario(Autos)
Control_esce        = escenario(Control)
df_indicadores_esce = pd.concat([Autos, Control])

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A 

In [12]:
#%% match de nuestros indicadores con el precio
comparacion_autos   = precios_df[precios_df.timestamp.isin(Autos.timestamp)]
comparacion_control = precios_df[precios_df.timestamp.isin(Control.timestamp)]

In [13]:
## comprehension de listas para data frames con 30 mins antes del comunicado y 30 mins despues del comunicado.
autoscom   = [precios_df.iloc[i-6:i+7] for i in comparacion_autos.index]
controlcom = [precios_df.iloc[i-6:i+7] for i in comparacion_autos.index]

In [14]:
Autos_esce['direccion']     = Autos['actual']
Autos_esce['pips alcistas'] = Autos['actual']
Autos_esce['pips bajistas'] = Autos['actual']
Autos_esce['volatilidad']   = Autos['actual']

Control_esce['direccion']     = Control['actual']
Control_esce['pips alcistas'] = Control['actual']
Control_esce['pips bajistas'] = Control['actual']
Control_esce['volatilidad']   = Control['actual']

In [15]:
def df_final(df, periodos):
    for i in range(len(df)):
        df['direccion'][i]     = periodos[i]["close"].iloc[-1] - periodos[i]["open"].iloc[6]
        df['pips alcistas'][i] = periodos[i]["high"].iloc[6:-1].max() - periodos[i]["open"].iloc[0]
        df['pips bajistas'][i] = periodos[i]["open"].iloc[0] - periodos[i]["low"].iloc[6:-1].min()
        df['volatilidad'][i]   = periodos[i]["high"].max()-periodos[i]["low"].min()
    return df

In [16]:
Final_autos   = df_final(Autos_esce, autoscom)
Final_control = df_final(Control_esce, controlcom)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A valu

In [17]:
# contamos ocurrencias por escenario
cont_esc_autos   = Final_autos['escenario'].value_counts().to_dict()
cont_esc_control = Final_control['escenario'].value_counts().to_dict()

In [18]:
trace = go.Bar(x=[cont_esc_autos,cont_esc_control])
data  = [trace]
py.iplot(data)


PlotlyRequestError: Authentication credentials were not provided.

### Conclusiones